In [1]:
# ライブラリの確認
import sys
from google.cloud import storage

print('python version: ', sys.version)
print("google cloud storage: ", storage.__version__)

python version:  3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)]
google cloud storage:  1.29.0


#### 認証情報取得

In [42]:
import os

# Credentialsの設定
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'project_folder/credential-gbqtoaml-1c3df6d8f54e.json'

# Credentialsの確認
print('Credentials from environ: {}'.format(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')))

Credentials from environ: project_folder/credential-gbqtoaml-1c3df6d8f54e.json


#### バケット作成

In [29]:
mybucketname = "mybucket-amlgcp202007"

In [30]:
from google.cloud import storage

def create_bucket(bucket_name):
    """Creates a new bucket."""
    # bucket_name = "your-new-bucket-name"

    storage_client = storage.Client()

    bucket = storage_client.create_bucket(bucket_name)

    print("Bucket {} created".format(bucket.name))

create_bucket(mybucketname)

Bucket mybucket-amlgcp202007 created


#### バケットの表示

In [20]:
from google.cloud import storage

def list_buckets():
    """Lists all buckets."""

    storage_client = storage.Client()
    buckets = storage_client.list_buckets()

    for bucket in buckets:
        print(bucket.name)
        
list_buckets()

bap-gbqtoaml-busket
mybucket-amlgcp2020


#### バケット情報の表示

In [31]:
import pprint
from google.cloud import storage

def bucket_metadata(bucket_name):
    """Prints out a bucket's metadata."""
    # bucket_name = 'your-bucket-name'

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    print("ID: {}".format(bucket.id))
    print("Name: {}".format(bucket.name))
    print("Storage Class: {}".format(bucket.storage_class))
    print("Location: {}".format(bucket.location))
    print("Location Type: {}".format(bucket.location_type))
    print("Cors: {}".format(bucket.cors))
    print(
        "Default Event Based Hold: {}".format(bucket.default_event_based_hold)
    )
    print("Default KMS Key Name: {}".format(bucket.default_kms_key_name))
    print("Metageneration: {}".format(bucket.metageneration))
    print(
        "Retention Effective Time: {}".format(
            bucket.retention_policy_effective_time
        )
    )
    print("Retention Period: {}".format(bucket.retention_period))
    print("Retention Policy Locked: {}".format(bucket.retention_policy_locked))
    print("Requester Pays: {}".format(bucket.requester_pays))
    print("Self Link: {}".format(bucket.self_link))
    print("Time Created: {}".format(bucket.time_created))
    print("Versioning Enabled: {}".format(bucket.versioning_enabled))
    print("Labels:")
    pprint.pprint(bucket.labels)
    
bucket_metadata(mybucketname)

ID: mybucket-amlgcp202007
Name: mybucket-amlgcp202007
Storage Class: STANDARD
Location: US
Location Type: multi-region
Cors: []
Default Event Based Hold: None
Default KMS Key Name: None
Metageneration: 1
Retention Effective Time: None
Retention Period: None
Retention Policy Locked: None
Requester Pays: False
Self Link: https://www.googleapis.com/storage/v1/b/mybucket-amlgcp202007
Time Created: 2020-07-08 05:06:47.371000+00:00
Versioning Enabled: False
Labels:
{}


#### オブジェクトをバケットにアップロードします。

In [33]:
from google.cloud import storage


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

upload_blob(mybucketname,
            "../data/breast_cancer.csv", 
            "data/breast_cancer.csv")

File ../data/breast_cancer.csv uploaded to data/breast_cancer.csv.


#### blobの表示

In [40]:
from google.cloud import storage

def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    for blob in blobs:
        print(blob.name)
        
list_blobs(mybucketname)

data/breast_cancer.csv


データの確認

In [47]:
import gcsfs
import pandas as pd

project_name = 'BAP-GBQtoAML-Dev'
file_path = mybucketname + '/data/breast_cancer.csv'

fs = gcsfs.GCSFileSystem(project=project_name,token=os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
fs.ls(mybucketname)

with fs.open(file_path, 'rb') as f:
    data = pd.read_csv(f)
data.head(2)

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
